# Generating the C Function for the $\psi_4$ Tetrad

### Author: Zach Etienne

<font color='red'>**This module has not yet undergone validation testing.**</font>

<a id='toc'></a>

# Introduction, Table of Contents
$$\label{toc}$$

In the [$\psi_4$ tutorial module](Tutorial-Psi4.ipynb), we constructed $\psi_4$ assuming an *arbitrary* tetrad. Then, given the Cartesian coordinate $(x,y,z)$ at which we want to evaluate $\psi_4$, the [tetrad module](../edit/BSSN/Psi4_tetrads.py) (**[tutorial](Tutorial-Psi4_tetrads.ipynb)**) outputs the tetrad in the $xx^i$ basis.

In this module, we will construct the C tetrad function. The C function works as follows:

1. Convert the BSSN grid coordinates $\left(xx_0,xx_1,xx_2\right)$ (given by the chosen reference metric [i.e., the chosen "reference_metric::CoordSystem" parameter]) to Cartesian coordinates $(x,y,z)$, which is the expected coordinate system for the tetrad.
1. Return the tetrad in the reference metric basis.

**This tutorial module is organized as follows:**

1. [Step 1](#initializenrpy): Initialize needed NRPy+ modules
1. [Step 2](#converttocartesian): Convert the coordinate location $\left(xx_0,xx_1,xx_2\right)$ at which we want the tetrad to the corresponding Cartesian coordinate $(x,y,z)$
1. [Step 3](#outtetrad): Output the tetrad in the reference-metric basis
1. [Step 4](#code_validation): Code Validation against BSSN.Psi4 NRPy+ module
1. [Step 5](#latex_pdf_output): Output this module to $\LaTeX$-formatted PDF

<a id='initializenrpy'></a>

# Step 1: Initialize core NRPy+ modules \[Back to [top](#toc)\]
$$\label{initializenrpy}$$

Let's start by importing all the needed modules from NRPy+:

In [1]:
# Step 1.a: import all needed modules from NRPy+:
import sympy as sp
from outputC import *
import NRPy_param_funcs as par
import indexedexp as ixp
import grid as gri
import finite_difference as fin
import reference_metric as rfm

# Step 1.b: Set the coordinate system for the numerical grid
par.set_parval_from_str("reference_metric::CoordSystem","Spherical")

# Step 1.c: Given the chosen coordinate system, set up 
#           corresponding reference metric and needed
#           reference metric quantities
# The following function call sets up the reference metric
#    and related quantities, including rescaling matrices ReDD,
#    ReU, and hatted quantities.
rfm.reference_metric()

# Step 1.d: Set spatial dimension (must be 3 for BSSN, as BSSN is 
#           a 3+1-dimensional decomposition of the general 
#           relativistic field equations)
DIM = 3

# Step 1.e: Import the tetrad module
import BSSN.Psi4_tetrads as BP4T
par.set_parval_from_str("BSSN.Psi4_tetrads::TetradChoice", "QuasiKinnersley")

<a id='converttocartesian'></a>

# Step 2: Convert the coordinate location $\left(xx_0,xx_1,xx_2\right)$ at which we want the tetrad to the corresponding Cartesian coordinate $(x,y,z)$ \[Back to [top](#toc)\]
$$\label{converttocartesian}$$

The C function uses as input $\left(xx_0,xx_1,xx_2\right)$, but the BSSN.Psi4_tetrads function uses $(x,y,z)$ as input. Thus the C function must first convert the coordinate $\left(xx_0,xx_1,xx_2\right)$ to the corresponding Cartesian coordinate.

In [2]:
# Step 2: Construct the C function header and 
#         convert (xx0,xx1,xx2) to the corresponding
#         (x,y,z), as both are needed for the tetrad
#         expressions.
outCparams = "preindent=1,outCfileaccess=a,outCverbose=False,includebraces=False"
tetrad_Ccode_filename = "BSSN/Psi4_tetrads.h"

with open(tetrad_Ccode_filename, "w") as file:
    file.write("""
// Taking as input (xx0,xx1,xx2), this function outputs
//   the chosen Psi4 tetrad in the (xx0,xx1,xx2) basis
void Psi4_tetrad(REAL xx0xx1xx2[3], const REAL *in_gfs,
                 REAL n4U[4],REAL mre4U[4],REAL mim4U[4]) {
    const REAL xx0 = xx0xx1xx2[0];
    const REAL xx1 = xx0xx1xx2[1];
    const REAL xx2 = xx0xx1xx2[2];
""")
outputC([rfm.xxCart[0],rfm.xxCart[1],rfm.xxCart[2]],["REAL x","REAL y","REAL z"],tetrad_Ccode_filename,outCparams)

Appended to file "BSSN/Psi4_tetrads.h"


<a id='outtetrad'></a>

# Step 3: Output the tetrad in the reference-metric basis \[Back to [top](#toc)\]
$$\label{outtetrad}$$

Next we call BP4T.Psi4_tetrads() to construct the symbolic expressions for the tetrad vectors $n^\mu$, $\Re m^\mu$, and $\Im m^\mu$, which are needed to construct $\Psi_4$. 

As the tetrad expressions depend on BSSN gridfunctions, we pass the expressions into fin.FD_outputC() so that the needed gridfunction values are read in from memory as appropriate.

In [3]:
# Step 3: Output the tetrad in the reference-metric basis.

# Step 3.a: BP4T.Psi4_tetrads() to construct the symbolic 
#           expressions for the tetrad vectors $n^\mu$, 
#           $\Re m^\mu$, and $\Im m^\mu$, which are needed 
#           to construct $\Psi_4$. 

BP4T.Psi4_tetrads()
Psi4_tetrad_vecs = []

# Step 3.b: As the tetrad expressions depend on BSSN 
#           gridfunctions, we pass the expressions into 
#           fin.FD_outputC() so that the needed gridfunction
#           values are read in from memory as appropriate.
for i in range(4):
    Psi4_tetrad_vecs.append(lhrh(lhs="n4U["+str(i)+"]",  rhs=BP4T.n4U[i]))
    Psi4_tetrad_vecs.append(lhrh(lhs="mre4U["+str(i)+"]",rhs=BP4T.mre4U[i]))
    Psi4_tetrad_vecs.append(lhrh(lhs="mim4U["+str(i)+"]",rhs=BP4T.mim4U[i]))

fin.FD_outputC(tetrad_Ccode_filename,Psi4_tetrad_vecs, outCparams)

with open(tetrad_Ccode_filename, "a") as file:
    file.write("}\n")

initialize_param() minor warning: Did nothing; already initialized parameter reference_metric::M_PI
initialize_param() minor warning: Did nothing; already initialized parameter reference_metric::RMAX
initialize_param() minor warning: Did nothing; already initialized parameter reference_metric::M_PI
initialize_param() minor warning: Did nothing; already initialized parameter reference_metric::RMAX
Appended to file "BSSN/Psi4_tetrads.h"


<a id='code_validation'></a>

# Step 4: Code validation against BSSN.Psi4_tetrads_Ccode_function NRPy+ module \[Back to [top](#toc)\]
$$\label{code_validation}$$

As a code validation check, we verify agreement in the SymPy expressions for the RHSs of the BSSN equations between
1. this tutorial and 
2. the NRPy+ BSSN.Psi4_tetrads_Ccode_function module.

By default, we compare the file that is output, assuming quantities are expressed in Spherical coordinates. Other coordinate systems may be chosen if desired.

In [4]:
import filecmp

import BSSN.Psi4_tetrads_Ccode_function as Psi4C
Psi4C.Psi4_tetrads_Ccode_function(tetrad_Ccode_filename="BSSN/Psi4_tetrads-validation.h")

if filecmp.cmp('BSSN/Psi4_tetrads.h',
               'BSSN/Psi4_tetrads-validation.h') == False:
    print("\nERROR: Validation test FAILED!")
    exit(1)
else:
    print("\nValidation test PASSED.")

initialize_param() minor warning: Did nothing; already initialized parameter reference_metric::M_PI
initialize_param() minor warning: Did nothing; already initialized parameter reference_metric::RMAX
Appended to file "BSSN/Psi4_tetrads-validation.h"
initialize_param() minor warning: Did nothing; already initialized parameter reference_metric::M_PI
initialize_param() minor warning: Did nothing; already initialized parameter reference_metric::RMAX
initialize_param() minor warning: Did nothing; already initialized parameter reference_metric::M_PI
initialize_param() minor warning: Did nothing; already initialized parameter reference_metric::RMAX
initialize_param() minor warning: Did nothing; already initialized parameter BSSN.Psi4_tetrads::x
initialize_param() minor warning: Did nothing; already initialized parameter BSSN.Psi4_tetrads::y
initialize_param() minor warning: Did nothing; already initialized parameter BSSN.Psi4_tetrads::z
Appended to file "BSSN/Psi4_tetrads-validation.h"

Valid

<a id='latex_pdf_output'></a>

# Step 5: Output this module to $\LaTeX$-formatted PDF file \[Back to [top](#toc)\]
$$\label{latex_pdf_output}$$

In [5]:
!jupyter nbconvert --to latex --template latex_nrpy_style.tplx Tutorial-Psi4_tetrads_Ccode_function.ipynb
!pdflatex -interaction=batchmode Tutorial-Psi4_tetrads_Ccode_function.tex
!pdflatex -interaction=batchmode Tutorial-Psi4_tetrads_Ccode_function.tex
!pdflatex -interaction=batchmode Tutorial-Psi4_tetrads_Ccode_function.tex
!rm -f Tut*.out Tut*.aux Tut*.log

[NbConvertApp] Converting notebook Tutorial-Psi4_tetrads_Ccode_function.ipynb to latex
[NbConvertApp] Writing 32617 bytes to Tutorial-Psi4_tetrads_Ccode_function.tex
This is pdfTeX, Version 3.14159265-2.6-1.40.18 (TeX Live 2017/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
This is pdfTeX, Version 3.14159265-2.6-1.40.18 (TeX Live 2017/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
This is pdfTeX, Version 3.14159265-2.6-1.40.18 (TeX Live 2017/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
